In [1]:
# create audio track
!ffmpeg -i ../LLM_valid_ru.mkv -vn -acodec pcm_s16le -ar 16000 -ac 1 ../LLM_valid_ru.wav


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/kolyangg/anaconda3/envs/stylegan3 --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, matroska,webm, from '../LLM_valid_ru.mkv':
  Metadata:
    COMPATIBLE_BRANDS: iso6avc1mp41
    MAJOR_BRAND     : dash
    MINOR_VER

In [4]:
# HF code to use Whisper

import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline


torch_dtype = torch.bfloat16 # set your preferred type here 

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
    setattr(torch.distributed, "is_initialized", lambda : False) # monkey patching
device = torch.device(device)

whisper = WhisperForConditionalGeneration.from_pretrained(
    "antony66/whisper-large-v3-russian", torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True,
    # add attn_implementation="flash_attention_2" if your GPU supports it
)

processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=256,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# read your wav file into variable wav. For example:

# wav = BytesIO()
# with open('../LLM_valid_ru.wav', 'rb') as f:
#     wav.write(f.read())
# wav.seek(0)

# # get the transcription
# asr = asr_pipeline(wav, generate_kwargs={"language": "russian", "max_new_tokens": 256}, return_timestamps=False)

# print(asr['text'])


# Read the WAV file into a NumPy array (and get its sample rate)
import soundfile as sf
wav, sr = sf.read('../LLM_valid_ru.wav')

# Optionally, check that your sample rate is what you expect (e.g. 16000 Hz)
print("Sample rate:", sr)

# Now pass the numpy array to the ASR pipeline
asr = asr_pipeline(wav, generate_kwargs={"language": "russian", "max_new_tokens": 256}, return_timestamps=False)
print(asr['text'])



Device set to use cuda
/home/kolyangg/anaconda3/envs/LLM/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sample rate: 16000
Подписывайтесь на наш канал.Подписывайтесь на наш канал.Мы не хотим, чтобы в мире были такие проблемы, как в мире, в котором есть такие проблемы.Подписывайтесь на наш канал.Подписывайтесь на наш канал.Все привет, давайте подождем и начнем.Подписывайтесь на наш канал.Да, давайте начнем. У нас сегодня ждет не то чтобы сильно большая, поэтому скорее всего мы проведем ее без перерыва. Думаю, мы успеем где-то за час максимум все это дело рассказать. Но при этом заключительная лекция в блоке первом лекции связанных с лоломками, такими именно больше коры историями про модальности и про то, как их обучают, какие там способы используют. И сегодня мы, по большей части, как раз поговорим про какие-то оставшиеся модальности, которые не успели затронуть на предыдущей лекции, и заодно немного поговорим о тех методах, которые используют для того, чтобы вообще сервис на базе лампы какой-то свой сделать, почему эти методы вдруг актуальны и зачем их используют, и подвернем некоторые в

In [ ]:
def transcribe_audio(
    input_file, 
    output_file, 
    batch_size=16, 
    cpu_cores=1, 
    language="russian", 
    return_timestamps=False,
    chunk_length=30,       # chunk length in seconds
    overlap_seconds=5      # overlap in seconds between consecutive chunks
):
    import time
    import numpy as np
    import torch
    import soundfile as sf
    from datasets import Dataset
    from transformers import (
        WhisperForConditionalGeneration, 
        WhisperProcessor, 
        pipeline,
        WhisperTimeStampLogitsProcessor
    )

    torch_dtype = torch.bfloat16

    # --- Load audio and split into overlapping chunks ---
    wav, sr = sf.read(input_file)
    print("Sample rate:", sr)
    duration = len(wav) / sr
    print(f"Audio duration: {duration:.2f} seconds")
    
    # Compute step size for sliding window
    step = chunk_length - overlap_seconds
    chunks = []
    start_times = []
    current_start = 0.0
    while current_start < duration:
        current_end = current_start + chunk_length
        start_idx = int(current_start * sr)
        end_idx = int(min(current_end * sr, len(wav)))
        chunks.append(wav[start_idx:end_idx])
        start_times.append(current_start)
        current_start += step

    data = {"audio": chunks, "start_time": start_times}
    dataset = Dataset.from_dict(data)

    # --- Helper function to process a batch of audio ---
    def transcribe_batch(batch, asr_pipeline_local, language):
        audio_list = batch["audio"]
        # Ensure each sample is a 1D float32 numpy array; pad them to the same length in the batch.
        max_len = max(len(a) for a in audio_list)
        padded_list = []
        for a in audio_list:
            a = np.array(a, dtype=np.float32).squeeze()
            if a.ndim != 1:
                raise ValueError("Expected a single channel (1D) audio array.")
            if len(a) < max_len:
                a = np.pad(a, (0, max_len - len(a)), mode="constant")
            padded_list.append(a)
        # Include the WhisperTimeStampLogitsProcessor in generate_kwargs.
        results = asr_pipeline_local(
            padded_list,
            generate_kwargs={
                "language": language, 
                "max_new_tokens": 256,
                "logits_processor": [WhisperTimeStampLogitsProcessor(asr_pipeline_local.model.generation_config)]
            }
        )
        if return_timestamps:
            transcriptions = []
            for r in results:
                if "chunks" in r and r["chunks"]:
                    chunk_strs = []
                    for ch in r["chunks"]:
                        # Get the timestamp list; if the ending timestamp is missing, output "NA"
                        ts = ch.get("timestamp", [None, None])
                        start_time_chunk = ts[0] if ts[0] is not None else 0.0
                        end_time_chunk = ts[1]
                        if end_time_chunk is None:
                            end_time_str = "NA"
                        else:
                            end_time_str = f"{end_time_chunk:.2f}"
                        chunk_strs.append(f"[{start_time_chunk:.2f}-{end_time_str}] {ch['text'].strip()}")
                    transcriptions.append(" ".join(chunk_strs))
                else:
                    transcriptions.append(r["text"].strip())
            batch["transcription"] = transcriptions
        else:
            batch["transcription"] = [r["text"].strip() for r in results]
        return batch

    # --- Worker function for multiprocessing on CPU ---
    def transcribe_worker(batch, language, batch_size):
        local_device = torch.device("cpu")
        model = WhisperForConditionalGeneration.from_pretrained(
            "antony66/whisper-large-v3-russian",
            torch_dtype=torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True,
        )
        processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")
        asr_pipeline_local = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            max_new_tokens=256,
            chunk_length_s=chunk_length,
            batch_size=batch_size,
            return_timestamps=return_timestamps,
            torch_dtype=torch_dtype,
            device=local_device,
        )
        return transcribe_batch(batch, asr_pipeline_local, language)

    # --- Decide processing mode (GPU or CPU multiprocess) ---
    if torch.cuda.is_available() or cpu_cores == 1:
        device_main = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = WhisperForConditionalGeneration.from_pretrained(
            "antony66/whisper-large-v3-russian",
            torch_dtype=torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True,
        )
        processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")
        asr_pipeline_global = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            max_new_tokens=256,
            chunk_length_s=chunk_length,
            batch_size=batch_size,
            return_timestamps=return_timestamps,
            torch_dtype=torch_dtype,
            device=device_main,
        )

        def transcribe_batch_main(batch):
            return transcribe_batch(batch, asr_pipeline_global, language)

        dataset = dataset.map(transcribe_batch_main, batched=True, batch_size=batch_size)
    else:
        dataset = dataset.map(
            transcribe_worker,
            batched=True,
            batch_size=batch_size,
            num_proc=cpu_cores,
            fn_kwargs={"language": language, "batch_size": batch_size}
        )

    # --- Combine transcriptions and write to output file ---
    # Write each chunk's transcription on a new line, along with its start time.
    lines = []
    for start, text in zip(dataset["start_time"], dataset["transcription"]):
        lines.append(f"[Chunk start {start:.2f}s]: {text}")
    full_transcription = "\n".join(lines)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(full_transcription)

    print("Transcription saved to", output_file)
    return full_transcription

# Example usage:
if __name__ == "__main__":
    transcribe_audio(
        input_file="../LLM_valid_ru.wav",
        output_file="transcription_timest.txt",
        batch_size=16,
        cpu_cores=16,  # adjust as desired
        language="russian", 
        return_timestamps=True,
        chunk_length=30,
        overlap_seconds=5
    )


Sample rate: 16000
Audio duration: 4555.26 seconds


Device set to use cuda


Map:   0%|          | 0/183 [00:00<?, ? examples/s]

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the m

Transcription saved to transcription_timest.txt


: 

## Incl. hot words

In [ ]:
def transcribe_audio(
    input_file, 
    output_file, 
    batch_size=16, 
    cpu_cores=1, 
    language="russian", 
    return_timestamps=False,
    chunk_length=30,       # chunk length in seconds
    overlap_seconds=5,     # overlap in seconds between consecutive chunks
    hot_words=None,        # list of hot words to bias (e.g. ["ООО", "ИП", "РФ"])
    hot_word_bias=5.0      # bias value added to logits for each hot word token
):
    import time
    import numpy as np
    import torch
    import soundfile as sf
    from datasets import Dataset
    from transformers import (
        WhisperForConditionalGeneration, 
        WhisperProcessor, 
        pipeline,
        WhisperTimeStampLogitsProcessor,
        LogitsProcessor
    )
    
    torch_dtype = torch.bfloat16
    
    # --- Load audio and split into overlapping chunks ---
    wav, sr = sf.read(input_file)
    print("Sample rate:", sr)
    duration = len(wav) / sr
    print(f"Audio duration: {duration:.2f} seconds")
    
    # Compute step size for sliding window
    step = chunk_length - overlap_seconds
    chunks = []
    start_times = []
    current_start = 0.0
    while current_start < duration:
        current_end = current_start + chunk_length
        start_idx = int(current_start * sr)
        end_idx = int(min(current_end * sr, len(wav)))
        chunks.append(wav[start_idx:end_idx])
        start_times.append(current_start)
        current_start += step
    
    data = {"audio": chunks, "start_time": start_times}
    dataset = Dataset.from_dict(data)
    
    # --- Define a custom logits processor for hot words ---
    class HotWordsBiasProcessor(LogitsProcessor):
        def __init__(self, hotwords_ids, bias_value=5.0):
            self.hotwords_ids = hotwords_ids
            self.bias_value = bias_value
        def __call__(self, input_ids, scores):
            # scores: tensor of shape (batch_size, vocab_size)
            for token_id in self.hotwords_ids:
                scores[:, token_id] += self.bias_value
            return scores
    
    # --- Helper function to process a batch of audio ---
    def transcribe_batch(batch, asr_pipeline_local, language):
        audio_list = batch["audio"]
        # Ensure each sample is a 1D float32 numpy array; pad them to the same length.
        max_len = max(len(a) for a in audio_list)
        padded_list = []
        for a in audio_list:
            a = np.array(a, dtype=np.float32).squeeze()
            if a.ndim != 1:
                raise ValueError("Expected a single channel (1D) audio array.")
            if len(a) < max_len:
                a = np.pad(a, (0, max_len - len(a)), mode="constant")
            padded_list.append(a)
        
        # Build the list of logits processors.
        logits_processors = []
        if return_timestamps:
            logits_processors.append(WhisperTimeStampLogitsProcessor(asr_pipeline_local.model.generation_config))
        if hot_words:
            # Convert each hot word to token id(s) using the pipeline's tokenizer.
            hotwords_ids = []
            for word in hot_words:
                token_ids = asr_pipeline_local.tokenizer.encode(word, add_special_tokens=False)
                if token_ids:
                    # Here we simply take the first token id.
                    hotwords_ids.append(token_ids[0])
            if hotwords_ids:
                logits_processors.append(HotWordsBiasProcessor(hotwords_ids, bias_value=hot_word_bias))
        
        results = asr_pipeline_local(
            padded_list,
            generate_kwargs={
                "language": language, 
                "max_new_tokens": 256,
                "logits_processor": logits_processors
            }
        )
        if return_timestamps:
            transcriptions = []
            for r in results:
                if "chunks" in r and r["chunks"]:
                    chunk_strs = []
                    for ch in r["chunks"]:
                        ts = ch.get("timestamp", [None, None])
                        start_time_chunk = ts[0] if ts[0] is not None else 0.0
                        end_time_chunk = ts[1]
                        if end_time_chunk is None:
                            end_time_str = "NA"
                        else:
                            end_time_str = f"{end_time_chunk:.2f}"
                        chunk_strs.append(f"[{start_time_chunk:.2f}-{end_time_str}] {ch['text'].strip()}")
                    transcriptions.append(" ".join(chunk_strs))
                else:
                    transcriptions.append(r["text"].strip())
            batch["transcription"] = transcriptions
        else:
            batch["transcription"] = [r["text"].strip() for r in results]
        return batch
    
    # --- Worker function for multiprocessing on CPU ---
    def transcribe_worker(batch, language, batch_size):
        local_device = torch.device("cpu")
        model = WhisperForConditionalGeneration.from_pretrained(
            "antony66/whisper-large-v3-russian",
            torch_dtype=torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True,
        )
        processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")
        asr_pipeline_local = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            max_new_tokens=256,
            chunk_length_s=chunk_length,
            batch_size=batch_size,
            return_timestamps=return_timestamps,
            torch_dtype=torch_dtype,
            device=local_device,
        )
        return transcribe_batch(batch, asr_pipeline_local, language)
    
    # --- Decide processing mode (GPU or CPU multiprocess) ---
    if torch.cuda.is_available() or cpu_cores == 1:
        device_main = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = WhisperForConditionalGeneration.from_pretrained(
            "antony66/whisper-large-v3-russian",
            torch_dtype=torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True,
        )
        processor = WhisperProcessor.from_pretrained("antony66/whisper-large-v3-russian")
        asr_pipeline_global = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            max_new_tokens=256,
            chunk_length_s=chunk_length,
            batch_size=batch_size,
            return_timestamps=return_timestamps,
            torch_dtype=torch_dtype,
            device=device_main,
        )
    
        def transcribe_batch_main(batch):
            return transcribe_batch(batch, asr_pipeline_global, language)
    
        dataset = dataset.map(transcribe_batch_main, batched=True, batch_size=batch_size)
    else:
        dataset = dataset.map(
            transcribe_worker,
            batched=True,
            batch_size=batch_size,
            num_proc=cpu_cores,
            fn_kwargs={"language": language, "batch_size": batch_size}
        )
    
    # --- Combine transcriptions and write to output file ---
    # Each chunk's transcription is written on a new line with its start time.
    lines = []
    for start, text in zip(dataset["start_time"], dataset["transcription"]):
        lines.append(f"[Chunk start {start:.2f}s]: {text}")
    full_transcription = "\n".join(lines)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(full_transcription)
    
    print("Transcription saved to", output_file)
    return full_transcription

# Example usage:
if __name__ == "__main__":
    transcribe_audio(
        input_file="../LLM_valid_ru.wav",
        output_file="transcription_timest.txt",
        batch_size=16,
        cpu_cores=16,  # adjust as desired
        language="russian", 
        return_timestamps=True,
        chunk_length=30,
        overlap_seconds=5,
        hot_words=["cross attention", "flash attention", "LLM", "throughput","swe bench", "human eval",  "vision", "kv-cache", 
                   "bigbenchaudio", "multi-line"]  # example hot words to bias the transcription
    )
